In [ ]:
# ==========================================
# File: 02_model_tuning_svm.ipynb (去重极简版·防止双重筛选)
# ==========================================

import os
import json
import pandas as pd
import numpy as np
from sklearn.svm import SVR
from sklearn.model_selection import RandomizedSearchCV, RepeatedKFold
from sklearn.preprocessing import PowerTransformer, RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import TransformedTargetRegressor
from scipy.stats import loguniform, uniform

# 1. 修正路径
if 'notebooks' in os.getcwd():
    os.chdir('..')

# 2. 读取数据 (只读取 70% 的训练集)
print(">>> [Step 1] Loading Training Data...")
try:
    df_train = pd.read_csv('data/processed/train_data.csv')
    print(f"✅ 成功加载训练集，样本数: {len(df_train)}")
    print(f"   当前特征数量: {df_train.shape[1] - 2} (已在 01 中经过 ElasticNet 筛选)")
except FileNotFoundError:
    raise FileNotFoundError("❌ 找不到 data/processed/train_data.csv！")

y_train = df_train['Target_Log1o2']
X_train = df_train.drop(columns=['Target_Log1o2', 'SMILES'])

# ========================================================
# 3. 搭建 SVM 纯净管道 (无 RFECV)
# ========================================================
print(">>> [Step 2] Building Pipeline (Pure SVM)...")

# 我们的策略：信任 01 号文件的筛选结果，这里不再做二次筛选
# 直接把数据喂给 SVM，依靠 SVM 自身的正则化能力来处理
inner_pipe = Pipeline([
    ('yeo', PowerTransformer()),  # 纠正数据偏态 (QSAR 数据必备)
    ('scaler', RobustScaler()),   # 抗异常值缩放
    ('svr', SVR())                # 最终模型
])

full_model = TransformedTargetRegressor(regressor=inner_pipe, transformer=RobustScaler())

# ========================================================
# 4. 定义参数空间
# ========================================================
param_dist = {
    # C: 惩罚力度。因为特征已经经过筛选，我们可以允许稍微大一点的 C
    # 但为了防止过拟合，上限控制在 100 左右是比较安全的
    'regressor__svr__C': loguniform(0.1, 100),

    # Gamma: 控制核函数的“管辖范围”。
    'regressor__svr__gamma': loguniform(0.001, 0.2),

    # Epsilon: 容忍度管道。对于 log(1o2) 这种数值，0.1~0.3 是合理的
    'regressor__svr__epsilon': uniform(0.05, 0.25)
}

# ========================================================
# 5. 开始搜索 (Repeated CV)
# ========================================================
print(">>> [Step 3] Running RandomizedSearchCV (Repeated K-Fold)...")
print("    (使用重复交叉验证来对抗小样本噪声)")

# 5折 * 10次重复 = 50 次验证，确保评分极其稳定
rkf = RepeatedKFold(n_splits=5, n_repeats=10, random_state=42)

search = RandomizedSearchCV(
    full_model,
    param_dist,
    n_iter=60,       # 搜索 60 组参数
    cv=rkf,          # 稳健 CV
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    random_state=42,
    verbose=1
)

search.fit(X_train, y_train)

# ========================================================
# 6. 结果分析与保存
# ========================================================
print(f"\n🏆 SVM Best Params: {search.best_params_}")
print(f"   Best CV Score (RMSE): {np.sqrt(-search.best_score_):.4f}")

# 保存
os.makedirs('results/models', exist_ok=True)
with open('results/models/best_params_svm.json', 'w') as f:
    json.dump(search.best_params_, f)

print("✅ 参数已保存！")
print("   请运行 03 进行最终测试！")